In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

pd.set_option('mode.chained_assignment', None)

# Participant Descriptors

In [ ]:
part_desc = pd.read_excel('Data/participant_descriptors_original.xlsx')
part_desc.dropna(axis='index', how='all', inplace=True)
part_desc.dropna(axis='columns', how='all', inplace=True)
part_desc.reset_index(inplace=True, drop=True)
part_desc.iloc[:, : ]

In [ ]:
part_desc.info()

In [ ]:
part_desc = part_desc.drop(['Dataset', 'Study', 'Initials'], axis=1)
part_desc.columns = ['sex', 'age(y)', 'bm(kg)', 'height(m)', 'vo2_peak(L.min-1)', 'peak_power(W)', 'get(L.min-1)', 'get(W)']
part_desc = part_desc.fillna(0.0)
part_desc.iloc[:, : ]

In [ ]:
print(part_desc['get(L.min-1)'].min())
print(part_desc['get(L.min-1)'].max())

def format_value(x):
    if isinstance(x, (int, float)):
        if x < 100:
            return float('{:.3f}'.format(x))
        else:
            return float('{:.3f}'.format(x / 1000))
    else:
        return x
    
part_desc['get(L.min-1)'] = part_desc['get(L.min-1)'].apply(format_value)

print(part_desc['get(L.min-1)'].min())
print(part_desc['get(L.min-1)'].max())

#1 is female, 2 is male
part_desc['sex'] = part_desc['sex'].replace({'F': 1, 'M': 2}).astype(int)
part_desc['age(y)'] = part_desc['age(y)'].astype(int)

In [ ]:
part_desc.hist(layout=(4, 4), figsize=(10, 10))
plt.tight_layout()
plt.show()

# Power

In [ ]:
power = pd.read_excel('Data/power_original.xlsx')
power.dropna(axis='index', how='all', inplace=True)
power.dropna(axis='columns', how='all', inplace=True)
power = power.iloc[:-1 , 1:]
power.reset_index(inplace=True, drop=True)
power.iloc[:, : ]

In [ ]:
power.info()

In [ ]:
power_trans = power.T
power_trans.reset_index(inplace=True, drop=True)
power_trans.iloc[:, : ]

In [ ]:
print('There are', power_trans.isnull().sum().sum(), 'missing values')

# Find empty cells
power_empty_rows = 0
for index, row in power_trans.iterrows():
    if row.isnull().all():
        power_empty_rows = cadence_empty_rows + 1
        continue
        
    # Fill empty values with the average of the preceding and succeeding non-empty values
    for i in range(len(row)-1, -1, -1):
        if pd.isnull(row[i]):
            j = i - 1
            while pd.isnull(row[j]):
                j -= 1
            row[i] = row[j]
    
    power_trans.iloc[index] = row
    
print('There are', (power_trans.isnull().sum().sum()) - (power_empty_rows * 180), 'missing values')
print('There are', power_empty_rows, 'empty rows')

In [ ]:
power_flatten_data = power_trans.to_numpy().flatten().astype(float)
plt.hist(power_flatten_data, bins=40)
plt.xlabel('Power')
plt.show()

# Cadence 

In [ ]:
cadence = pd.read_excel('Data/cadence_original.xlsx')
cadence.dropna(axis='index', how='all', inplace=True)
cadence.dropna(axis='columns', how='all', inplace=True)
cadence = cadence.iloc[:-1 , 1:]
cadence.reset_index(inplace=True, drop=True)
cadence.iloc[:, : ]

In [ ]:
cadence.info()

In [ ]:
cadence_trans = cadence.T
cadence_trans.reset_index(inplace=True, drop=True)
cadence_trans.iloc[:, : ]

In [ ]:
print('There are', cadence_trans.isnull().sum().sum(), 'missing values')

# Find empty cells
cadence_empty_rows = 0
for index, row in cadence_trans.iterrows():
    if row.isnull().all():
        cadence_empty_rows = cadence_empty_rows + 1
        continue
        
    # Fill empty values with the average of the preceding and succeeding non-empty values
    for i in range(len(row)-1, -1, -1):
        if pd.isnull(row[i]):
            j = i - 1
            while pd.isnull(row[j]):
                j -= 1
            row[i] = row[j]
    
    cadence_trans.iloc[index] = row
    
print('There are', (cadence_trans.isnull().sum().sum()) - (cadence_empty_rows * 180), 'missing values')
print('There are', cadence_empty_rows, 'empty rows')

In [ ]:
cadence_flatten_data = cadence_trans.to_numpy().flatten().astype(float)
plt.xlabel('Cadence')
plt.hist(cadence_flatten_data, bins=40)
plt.show()

# VO2

In [ ]:
vo2 = pd.read_excel('Data/vo2_original.xlsx')
cadence.dropna(axis='index', how='all', inplace=True)
cadence.dropna(axis='columns', how='all', inplace=True)
vo2 = vo2.iloc[: , 1:]
vo2.reset_index(inplace=True, drop=True)
vo2.iloc[:, : ]

In [ ]:
vo2.info()

In [ ]:
vo2_trans = vo2.T.astype(float)
vo2_trans.reset_index(inplace=True, drop=True)
vo2_trans.iloc[:, : ]

In [ ]:
print(vo2_trans.min().min())
print(vo2_trans.max().max())

def format_value(x):
    if isinstance(x, (int, float)):
        if x < 100 and x > 0:
            return float('{:.3f}'.format(x))
        elif x > 100:
            return float('{:.3f}'.format(x / 1000))
        else:
            return x
    else:
        return x
    
vo2_trans = vo2_trans.applymap(format_value)

print(vo2_trans.min().min())
print(vo2_trans.max().max())

In [ ]:
vo2_flatten_data = vo2_trans.to_numpy().flatten().astype(float)
plt.hist(vo2_flatten_data, bins=40)
plt.xlabel('VO2')
plt.show()

In [ ]:
indices = range(9, 180, 10) 

# Contains only rows 10, 20, 30, ...
vo2_trans_2 = vo2_trans.iloc[:, indices]

x_values = vo2_trans_2[130:131].values
y_values = vo2_trans_2.columns

plt.scatter(y_values, x_values)
plt.xlabel('Time (s)')
plt.ylabel('VO2')
plt.ylim(0, 5)
plt.show()

In [ ]:
# Find empty cells
vo2_empty_rows = 0
for index, row in vo2_trans_2.iterrows():
    if row.isnull().all():
        vo2_empty_rows += 1
        continue
        
    # Fill empty values with the average of the preceding and succeeding non-empty values
    for i in range(179, 8, -10):
        if pd.isnull(row[i]):
            j = i
            while j > 9 and pd.isnull(row[j]):
                j -= 10
                if pd.notnull(row[j]):
                    row[i] = row[j]
            if j <= 9:
                j = i
                j += 10
                if pd.notnull(row[j]):
                    row[i] = row[j]
                    
    vo2_trans_2_copy = vo2_trans_2.copy()
    vo2_trans_2_copy.loc[index] = row
    
print('There are', (vo2_trans_2_copy.isnull().sum().sum()) - (vo2_empty_rows * 18), 'missing values')
print('There are', vo2_empty_rows, 'empty rows')

#Fills empty rows with 0s
vo2_trans_2_copy = vo2_trans_2_copy.fillna(0)

In [ ]:
# Interpolate vo2
new_columns = list(range(0, 180))
vo2_interpolated = vo2_trans_2_copy.reindex(columns=new_columns).interpolate(method='linear', axis=1)
vo2_interpolated.loc[:, :1] = vo2_interpolated.loc[:, :1].fillna(0)
vo2_interpolated.interpolate(method='linear', axis=1, inplace = True)

In [ ]:
vo2_flatten_data = vo2_interpolated.to_numpy().flatten().astype(float)

# plot a histogram of all the data
plt.hist(vo2_flatten_data, bins=40)
plt.xlabel('VO2')
plt.show()

# CP & W'

In [ ]:
power_cp = power_trans.iloc[:, -30:]
cp = power_cp.mean(axis=1)
w = (power_trans.sum(axis=1) - cp*180)/1000

# Add CP and W' to part_desc df
part_desc_complete = part_desc.copy()
part_desc_complete['CP'] = cp
part_desc_complete['w'] = w

# Add CP and W' to power df
power_trans_complete = power_trans.copy()
power_trans_complete.columns = power_trans_complete.columns.astype(str)
power_trans_complete.columns = 'power_' + power_trans_complete.columns
power_trans_complete['CP'] = cp
power_trans_complete['w'] = w

# Add CP and W' to cadence df
cadence_trans_complete = cadence_trans.copy()
cadence_trans_complete.columns = cadence_trans_complete.columns.astype(str)
cadence_trans_complete.columns = 'cadence_' + cadence_trans_complete.columns
cadence_trans_complete['CP'] = cp
cadence_trans_complete['w'] = w

# Add CP and W' to vo2 df
vo2_trans_complete = vo2_interpolated.copy()
vo2_trans_complete.columns = vo2_trans_complete.columns.astype(str)
vo2_trans_complete.columns = 'vo2_' + vo2_trans_complete.columns
vo2_trans_complete['CP'] = cp
vo2_trans_complete['w'] = w

In [ ]:
median_values = power_trans.median(axis=1)
mean_values = power_trans.mean(axis=1)

ordered_cp = cp.sort_values(ascending=False)
matched_median_values = median_values.loc[ordered_cp.index]
matched_mean_values = mean_values.loc[ordered_cp.index]

plt.plot(ordered_cp.values, label='CP', color='orange')
plt.plot(matched_median_values.values, label='Median Power', color='red')
plt.plot(matched_mean_values.values, label='Mean Power', color='blue')
plt.xlabel('Participant ID') 
plt.ylabel('Power')
plt.legend()
plt.show()

## Correlations

In [ ]:
corr = part_desc_complete.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
corr1 = power_trans_complete.corr()
corr1.style.background_gradient(cmap='coolwarm')

In [ ]:
corr2 = cadence_trans_complete.corr()
corr2.style.background_gradient(cmap='coolwarm')

In [ ]:
corr3 = vo2_trans_complete.corr()
corr3.style.background_gradient(cmap='coolwarm')

In [ ]:
# Column names
part_desc_complete_altered = part_desc_complete[~(part_desc_complete == 0).any(axis=1)]
column_names = part_desc_complete_altered.columns.tolist()
column_names.remove('CP')

# Plot each column against 'CP' 
for column in column_names:
    plt.figure(figsize=(8, 6))
    plt.scatter(part_desc_complete_altered['CP'], part_desc_complete_altered[column], label=column)
    plt.xlabel('CP')
    plt.ylabel(column)
    plt.title(f'Plot of {column} vs CP')
    plt.show()

In [ ]:
# Column names
column_names = part_desc_complete_altered.columns.tolist()
column_names.remove('w')

# Plot each column against 'w' 
for column in column_names:
    plt.figure(figsize=(8, 6))
    plt.scatter(part_desc_complete_altered['w'], part_desc_complete_altered[column], label=column)
    plt.xlabel("W'")
    plt.ylabel(column)
    plt.title(f"Plot of {column} vs W'")
    plt.show()

# Feature Engineer

### Power cumulative sum

In [ ]:
power_cumulative_df = power_trans_complete.copy()
power_cumulative_df.columns = 'cumulative_' + power_cumulative_df.columns

# Save CP and W values
CP_and_W_p = power_cumulative_df.iloc[:,-2:]

# Calculate gradients for each participant (row)
power_cumulative_df = power_cumulative_df.iloc[:,:-2].cumsum(axis=1)
power_cumulative_df = power_cumulative_df.fillna(0)
power_cumulative_df['CP'] = cp
power_cumulative_df['w'] = w

### Power derivative

In [ ]:
# Calculate derivatives (gradients) for each participant
power_derivatives_df = power_cumulative_df.iloc[:, :-2].diff(axis=1)
power_derivatives_df.columns = 'derivative_of_' + power_derivatives_df.columns
power_derivatives_df['CP'] = cp
power_derivatives_df['w'] = w

### Cadence cumulative sum

In [ ]:
cadence_cumulative_df = cadence_trans_complete.copy()
cadence_cumulative_df.columns = 'cumulative_' + cadence_cumulative_df.columns

# Save CP and W values
CP_and_W_p = cadence_cumulative_df.iloc[:,-2:]

# Calculate gradients for each participant (row)
cadence_cumulative_df = cadence_cumulative_df.iloc[:,:-2].cumsum(axis=1)
cadence_cumulative_df = cadence_cumulative_df.fillna(0)
cadence_cumulative_df['CP'] = cp
cadence_cumulative_df['w'] = w

### Cadence derivative

In [ ]:
# Calculate derivatives (gradients) for each participant
cadence_derivatives_df = cadence_cumulative_df.iloc[:, :-2].diff(axis=1)
cadence_derivatives_df.columns = 'derivative_of_' + cadence_derivatives_df.columns
cadence_derivatives_df['CP'] = cp
cadence_derivatives_df['w'] = w

### VO2 cumulative sum

In [ ]:
vo2_cumulative_df = vo2_trans_complete.copy()
vo2_cumulative_df.columns = 'cumulative_' + vo2_cumulative_df.columns

# Save CP and W values
CP_and_W_p = vo2_cumulative_df.iloc[:,-2:]

# Calculate gradients for each participant (row)
vo2_cumulative_df = vo2_cumulative_df.iloc[:,:-2].cumsum(axis=1)
vo2_cumulative_df = vo2_cumulative_df.fillna(0)
vo2_cumulative_df['CP'] = cp
vo2_cumulative_df['w'] = w

### VO2 derivative

In [ ]:
# Calculate derivatives (gradients) for each participant
vo2_derivatives_df = vo2_cumulative_df.iloc[:, :-2].diff(axis=1)
vo2_derivatives_df.columns = 'derivative_of_' + vo2_derivatives_df.columns

vo2_derivatives_df['CP'] = cp
vo2_derivatives_df['w'] = w

# Group rows by participant 

In [ ]:
part_desc_grouped = part_desc_complete.copy()
part_desc_grouped = part_desc_grouped[part_desc_grouped['vo2_peak(L.min-1)'] != 0]
part_desc_grouped['Individual'] = pd.factorize(part_desc_grouped['sex'].astype(str) + part_desc_grouped['age(y)'].astype(str) + part_desc_grouped['bm(kg)'].astype(str) + part_desc_grouped['height(m)'].astype(str) + part_desc_grouped['vo2_peak(L.min-1)'].astype(str) + part_desc_grouped['peak_power(W)'].astype(str) + part_desc_grouped['get(L.min-1)'].astype(str))[0] + 1
part_desc_grouped = part_desc_grouped[['Individual', 'sex', 'age(y)', 'bm(kg)', 'height(m)', 'vo2_peak(L.min-1)', 'peak_power(W)', 'get(L.min-1)', 'get(W)', 'CP', 'w']]
part_desc_ready = part_desc_grouped[~(part_desc_grouped == 0).any(axis=1)]
individuals = part_desc_grouped['Individual']

In [ ]:
power_trans_grouped = power_trans_complete.copy()
power_trans_grouped = power_trans_grouped.loc[~(power_trans_grouped == 0).all(axis=1)]
power_trans_grouped.insert(0, 'Individual', individuals)

In [ ]:
cadence_trans_grouped = cadence_trans_complete.copy()
cadence_trans_grouped = cadence_trans_grouped.loc[~(cadence_trans_grouped == 0).all(axis=1)]
cadence_trans_grouped.insert(0, 'Individual', individuals)

In [ ]:
vo2_trans_grouped = vo2_trans_complete.copy()
vo2_trans_grouped = vo2_trans_grouped.loc[~(vo2_trans_grouped.iloc[:,1:-2] == 0).all(axis=1)]
vo2_trans_grouped.insert(0, 'Individual', individuals)

In [ ]:
power_cumulative_grouped = power_cumulative_df.copy()
power_cumulative_grouped = power_cumulative_grouped.loc[~(power_cumulative_grouped == 0).all(axis=1)]
power_cumulative_grouped.insert(0, 'Individual', individuals)

In [ ]:
power_derivative_grouped = power_derivatives_df.copy()
power_derivative_grouped = power_derivative_grouped.loc[~(power_derivative_grouped == 0).all(axis=1)]
power_derivative_grouped.insert(0, 'Individual', individuals)

In [ ]:
cadence_derivative_grouped = cadence_derivatives_df.copy()
cadence_derivative_grouped = cadence_derivative_grouped.loc[~(cadence_derivative_grouped == 0).all(axis=1)]
cadence_derivative_grouped.insert(0, 'Individual', individuals)

In [ ]:
vo2_derivative_grouped = vo2_derivatives_df.copy()
vo2_derivative_grouped = vo2_derivative_grouped.loc[~(vo2_derivative_grouped == 0).all(axis=1)]
vo2_derivative_grouped.insert(0, 'Individual', individuals)

## Participant Descriptors

part_desc_ready

In [ ]:
part_desc_ready

## Number of seconds needed

In [ ]:
secs = -122

## Power

In [ ]:
# Save CP and W values
CP_and_W_p = power_trans_grouped.iloc[:,-2:]

# Select how many seconds are needed for power
power = power_trans_grouped.iloc[:,:secs]

# Add CP and W back to df
power = power.join(CP_and_W_p)

## Power Peaks

In [ ]:
power_peak = power.iloc[:,:-2].max(axis=1)
power_peak = pd.DataFrame(power_peak, columns=['Power_Peak'])

In [ ]:
# Extract the columns
column1 = cp
column2 = power_peak.iloc[:, 0]

# Create a scatter plot
plt.scatter(column1, column2)
plt.xlabel('Column from df1')
plt.ylabel('Column from df2')
plt.title('Scatter Plot: df1 vs df2')
plt.show()

## Cadence

In [ ]:
# Save CP and W values
CP_and_W_c = cadence_trans_grouped.iloc[:,-2:]

# Select how many seconds are needed for cadence
cadence = cadence_trans_grouped.iloc[:,:secs]

# Add CP and W back to df
cadence = cadence.join(CP_and_W_c)

## Cadence Peaks

In [ ]:
cadence_peak = cadence.iloc[:,:-2].max(axis=1)
cadence_peak = pd.DataFrame(cadence_peak, columns=['Cadence_Peak'])

In [ ]:
# Extract the columns
column1 = cadence.iloc[:,-2:-1]
column2 = cadence_peak.iloc[:, 0]

# Create a scatter plot
plt.scatter(column1, column2)
plt.xlabel('CP')
plt.ylabel('Cadence Peak')
plt.title('Scatter Plot: df1 vs df2')
plt.show()

## VO2


In [ ]:
# Save CP and W values
CP_and_W_v = vo2_trans_grouped.iloc[:,-2:]

# Select how many seconds are needed for vo2
vo2 = vo2_trans_grouped.iloc[:,:secs]

# Add CP and W back to df
vo2 = vo2.join(CP_and_W_v)

## VO2 Peaks

In [ ]:
vo2_peak = vo2.iloc[:,:-2].max(axis=1)
vo2_peak = pd.DataFrame(vo2_peak, columns=['VO2_Peak'])

In [ ]:
# Extract the columns
column1 = vo2.iloc[:,-2:-1]
column2 = vo2_peak.iloc[:, 0]

# Create a scatter plot
plt.scatter(column1, column2)
plt.xlabel('CP')
plt.ylabel('VO2 Peak')
plt.title('Scatter Plot: CP vs VO2_Peak')
plt.show()

## Participant Descriptors and Power

In [ ]:
# Join df
part_desc_power_joined = part_desc_ready.iloc[:,:-2].join(power_trans_grouped.iloc[:,1:])
part_desc_power_joined = part_desc_power_joined.dropna(how='any')

# Save CP and W values
CP_and_W_pdp = part_desc_power_joined.iloc[:,-2:]

# Select how many seconds are needed
part_desc_power_joined = part_desc_power_joined.iloc[:,:secs]

# Add CP and W back to df
part_desc_power_joined = part_desc_power_joined.join(CP_and_W_pdp)

## Participant Descriptors, Power, and Cadence

In [ ]:
# Join df and select how many seconds are needed
part_desc_power_cadence_joined = part_desc_ready.iloc[:,:-2].join(power_trans_grouped.iloc[:,1:secs]).join(cadence_trans_grouped.iloc[:,1:])
part_desc_power_cadence_joined = part_desc_power_cadence_joined.dropna(how='any')

# Save CP and W values
CP_and_W_pdpc = part_desc_power_cadence_joined.iloc[:,-2:]

# Select how many seconds are needed
part_desc_power_cadence_joined = part_desc_power_cadence_joined.iloc[:,:secs]

# Add CP and W back to df
part_desc_power_cadence_joined = part_desc_power_cadence_joined.join(CP_and_W_pdpc)

## Participant Descriptors, Power, Cadence, and VO2

In [ ]:
# Join df and select how many seconds are needed
part_desc_power_cadence_vo2_joined = part_desc_ready.iloc[:,:-2].join(power_trans_grouped.iloc[:,1:secs]).join(cadence_trans_grouped.iloc[:,1:secs]).join(vo2_trans_grouped.iloc[:,1:])
part_desc_power_cadence_vo2_joined = part_desc_power_cadence_vo2_joined.dropna(how='any')

# Save CP and W values
CP_and_W_pdpcv = part_desc_power_cadence_vo2_joined.iloc[:,-2:]

# Select how many seconds are needed
part_desc_power_cadence_vo2_joined = part_desc_power_cadence_vo2_joined.iloc[:,:secs]

# Add CP and W back to df
part_desc_power_cadence_vo2_joined = part_desc_power_cadence_vo2_joined.join(CP_and_W_pdpcv)

## Participant Descriptors, Power, Power Cumulative Sum, Cadence, VO2

In [ ]:
# Join df and select how many seconds are needed
part_desc_power_variants_cadence_vo2_joined = part_desc_ready.iloc[:,:-2].join(power_trans_grouped.iloc[:,1:secs]).join(power_cumulative_grouped.iloc[:,1:secs]).join(cadence_trans_grouped.iloc[:,1:secs]).join(vo2_trans_grouped.iloc[:,1:])
part_desc_power_variants_cadence_vo2_joined = part_desc_power_variants_cadence_vo2_joined.dropna(how='any')

# Save CP and W values
CP_and_W_pdpcv = part_desc_power_variants_cadence_vo2_joined.iloc[:,-2:]

# Select how many seconds are needed
part_desc_power_variants_cadence_vo2_joined = part_desc_power_variants_cadence_vo2_joined.iloc[:,:secs]

# Add CP and W back to df
part_desc_power_variants_cadence_vo2_joined = part_desc_power_variants_cadence_vo2_joined.join(CP_and_W_pdpcv)

## Participant Descriptors, Power, Peaks, Power and Cadence Derivatives

In [ ]:
# Join df and select how many seconds are needed
part_desc_peaks_derivatives_joined = part_desc_ready.iloc[:,:-2].join(power_trans_grouped.iloc[:,1:secs]).join(power_peak).join(cadence_peak).join(vo2_peak).join(power_derivative_grouped.iloc[:,2:secs]).join(cadence_derivative_grouped.iloc[:,2:])
part_desc_peaks_derivatives_joined = part_desc_peaks_derivatives_joined.dropna(how='any')

# Save CP and W values
CP_and_W_pdpd = part_desc_peaks_derivatives_joined.iloc[:,-2:]

# Select how many seconds are needed
part_desc_peaks_derivatives_joined = part_desc_peaks_derivatives_joined.iloc[:,:secs]

# Add CP and W back to df
part_desc_peaks_derivatives_joined = part_desc_peaks_derivatives_joined.join(CP_and_W_pdpd)

# Functions

### Train / Test  split

In [ ]:
def train_test_split(input_df, target):
    # Shuffle the DataFrame to randomize the order
    shuffling_df = input_df.sample(frac=1, random_state=42)

    # Identify individuals and the number of tests they have done
    individuals_counts_df = shuffling_df['Individual'].value_counts()

    # Create an empty list to store individuals for train and test splits
    train_individuals_df = []
    test_individuals_df = []

    # Iterate through individuals and add them to the splits while keeping individuals together
    for individual in individuals_counts_df.index:
        if len(train_individuals_df) / len(shuffling_df) < 0.7:
            train_individuals_df.extend([individual] * individuals_counts_df[individual])
        else:
            test_individuals_df.extend([individual] * individuals_counts_df[individual])

    # Create the final train and test DataFrames by selecting all rows corresponding to the selected individuals
    train_df = shuffling_df[shuffling_df['Individual'].isin(train_individuals_df)]
    test_df = shuffling_df[shuffling_df['Individual'].isin(test_individuals_df)]
    
    # Create X_train, y_train, X_test, y_test
    X_train = train_df.iloc[:,1:-2]
    y_train = train_df[[target]]
    X_test = test_df.iloc[:,1:-2]
    y_test = test_df[[target]]
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, y_train, X_test_scaled, y_test

### Baseline Regression

In [ ]:
def baseline(X_train, y_train, X_test, y_test):
    baseline_model = DummyRegressor(strategy='mean')
    baseline_model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred_baseline = baseline_model.predict(X_test)
    y_test_baseline = y_test.iloc[:, 0]

    # Evaluate the model
    mse = mean_squared_error(y_test_baseline, y_pred_baseline)
    r2 = r2_score(y_test_baseline, y_pred_baseline)
    mape = np.mean(np.abs((y_test_baseline - y_pred_baseline) / y_test_baseline)) * 100
    
    return mse, r2, mape

### Linear Regression

In [ ]:
def linear_regression(X_train, y_train, X_test, y_test):
    linear_model = LinearRegression()
    
    # Perform cross-validation
    mse_cv = -cross_val_score(linear_model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
    r2_cv = cross_val_score(linear_model, X_train, y_train, scoring='r2', cv=5)
    mape_cv = cross_val_score(linear_model, X_train, y_train, scoring='neg_mean_absolute_percentage_error', cv=5)
    
    # Fit the model
    linear_model.fit(X_train, y_train)
    
    y_pred_linear_test = linear_model.predict(X_test)
    y_test_linear_test = y_test.values.ravel()
    
    mse_test = mean_squared_error(y_test_linear_test, y_pred_linear_test)
    r2_test = r2_score(y_test_linear_test, y_pred_linear_test)
    mape_test = np.mean(np.abs((y_test_linear_test - y_pred_linear_test) / y_test_linear_test)) * 100
    
    return mse_test, r2_test, mape_test, np.mean(mse_cv), np.mean(r2_cv), -np.mean(mape_cv)

### Decision Tree Regression

In [ ]:
def decision_tree_regression(X_train, y_train, X_test, y_test):
    decision_tree_model = DecisionTreeRegressor(random_state=42)
    
    # Perform cross-validation
    mse_cv = -cross_val_score(decision_tree_model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
    r2_cv = cross_val_score(decision_tree_model, X_train, y_train, scoring='r2', cv=5)
    mape_cv = cross_val_score(decision_tree_model, X_train, y_train, scoring='neg_mean_absolute_percentage_error', cv=5)

    # Fit the model
    decision_tree_model.fit(X_train, y_train)
    
    y_pred_decision_tree_test = decision_tree_model.predict(X_test)
    y_test_decision_tree_test = y_test.values.ravel()

    mse_test = mean_squared_error(y_test_decision_tree_test, y_pred_decision_tree_test)
    r2_test = r2_score(y_test_decision_tree_test, y_pred_decision_tree_test)
    mape_test = np.mean(np.abs((y_test_decision_tree_test - y_pred_decision_tree_test) / y_test_decision_tree_test)) * 100
    
    return mse_test, r2_test, mape_test, np.mean(mse_cv), np.mean(r2_cv), -np.mean(mape_cv)

### Random Forest Regression

In [ ]:
def random_forest_regression(X_train, y_train, X_test, y_test):
    random_forest_model = RandomForestRegressor(n_estimators=150, max_depth=10, random_state=42)
    
    # Perform cross-validation
    mse_cv = -cross_val_score(random_forest_model, X_train, y_train.values.ravel(), scoring='neg_mean_squared_error', cv=5)
    r2_cv = cross_val_score(random_forest_model, X_train, y_train.values.ravel(), scoring='r2', cv=5)
    mape_cv = cross_val_score(random_forest_model, X_train, y_train.values.ravel(), scoring='neg_mean_absolute_percentage_error', cv=5)

    # Fit the model
    random_forest_model.fit(X_train, y_train.values.ravel())
    
    y_pred_random_forest_test = random_forest_model.predict(X_test)
    y_test_random_forest_test = y_test.values.ravel()

    mse_test = mean_squared_error(y_test_random_forest_test, y_pred_random_forest_test)
    r2_test = r2_score(y_test_random_forest_test, y_pred_random_forest_test)
    mape_test = np.mean(np.abs((y_test_random_forest_test - y_pred_random_forest_test) / y_test_random_forest_test)) * 100
    
    return mse_test, r2_test, mape_test, np.mean(mse_cv), np.mean(r2_cv), -np.mean(mape_cv)

## Only use for hyperparameter tuning

In [ ]:
# def decision_tree_regression(X_train, y_train, X_test, y_test):
#     # Define hyperparameters to tune
#     param_grid = {
#         'max_depth': [None, 5, 10, 15],
#         'min_samples_leaf': [1, 5, 10]
#     }

#     # Create Decision Tree model
#     decision_tree_model = DecisionTreeRegressor(random_state=42)

#     # Perform grid search with cross-validation
#     grid_search = GridSearchCV(decision_tree_model, param_grid, cv=5, scoring='neg_mean_squared_error')
#     grid_search.fit(X_train, y_train)

#     # Get the best hyperparameters
#     best_params = grid_search.best_params_
#     print('Best Hyperparameters:', best_params)
    
#     # Use the best model for predictions
#     best_decision_tree_model = grid_search.best_estimator_
    
#     # Perform cross-validation
#     mse_cv = -cross_val_score(best_decision_tree_model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
#     r2_cv = cross_val_score(best_decision_tree_model, X_train, y_train, scoring='r2', cv=5)
#     mape_cv = cross_val_score(best_decision_tree_model, X_train, y_train, scoring='neg_mean_absolute_percentage_error', cv=5)

#     # Use the best model for predictions
#     y_pred_decision_tree_test = best_decision_tree_model.predict(X_test)
#     y_test_decision_tree_test = y_test.values.ravel()

#     mse_test = mean_squared_error(y_test_decision_tree_test, y_pred_decision_tree_test)
#     r2_test = r2_score(y_test_decision_tree_test, y_pred_decision_tree_test)
#     mape_test = np.mean(np.abs((y_test_decision_tree_test - y_pred_decision_tree_test) / y_test_decision_tree_test)) * 100
    
#     return mse_test, r2_test, mape_test, np.mean(mse_cv), np.mean(r2_cv), -np.mean(mape_cv)

In [ ]:
# def random_forest_regression(X_train, y_train, X_test, y_test):
#    # Hyperparameters to tune
#     param_grid = {
#         'n_estimators': [50, 100, 200],
#         'max_depth': [5, 10, 15, None],
#         'min_samples_leaf': [1, 5, 10]
#     }

    
#     # Create Random Forest model
#     random_forest_model = RandomForestRegressor(random_state=42)

#     # Perform grid search with cross-validation
#     grid_search = GridSearchCV(random_forest_model, param_grid, cv=5, scoring='neg_mean_squared_error')
#     grid_search.fit(X_train, y_train.values.ravel())

#     # Get the best hyperparameters
#     best_params = grid_search.best_params_
#     print('Best Hyperparameters:', best_params)
    
#     best_random_forest_model = grid_search.best_estimator_
    
#     # Perform cross-validation
#     mse_cv = -cross_val_score(best_random_forest_model, X_train, y_train.values.ravel(), scoring='neg_mean_squared_error', cv=5)
#     r2_cv = cross_val_score(best_random_forest_model, X_train, y_train.values.ravel(), scoring='r2', cv=5)
#     mape_cv = cross_val_score(best_random_forest_model, X_train, y_train.values.ravel(), scoring='neg_mean_absolute_percentage_error', cv=5)
    
#     # Use the best model for predictions
#     y_pred_random_forest_test = best_random_forest_model.predict(X_test)
#     y_test_random_forest_test = y_test.values.ravel()

#     mse_test = mean_squared_error(y_test_random_forest_test, y_pred_random_forest_test)
#     r2_test = r2_score(y_test_random_forest_test, y_pred_random_forest_test)
#     mape_test = np.mean(np.abs((y_test_random_forest_test - y_pred_random_forest_test) / y_test_random_forest_test)) * 100
    
#     return mse_test, r2_test, mape_test, np.mean(mse_cv), np.mean(r2_cv), -np.mean(mape_cv)

### Results

In [ ]:
# # Use to print results as the function below runs
# def print_results(cv, dataset_name, model_name, target, *args):
#     if cv == 'Y':
#         mse_test, r2_test, mape_test, mse_cv, r2_cv, mape_cv = args
#         print(f'{dataset_name} - {model_name} - {target} - Cross-Validation:')
#         print('Mean Squared Error (CV):', mse_cv)
#         print('R-squared (CV):', r2_cv)
#         print('Mean Absolute Percentage Error (MAPE) (CV):', mape_cv)
#         print()
#         print(f'{dataset_name} - {model_name} - {target} - TEST:')
#         print('Mean Squared Error (TEST):', mse_test)
#         print('R-squared (TEST):', r2_test)
#         print('Mean Absolute Percentage Error (MAPE) (TEST):', mape_test)
#         print()
#         print('---------------------------------------------------------------')
#         print()
#     else:
#         mse, r2, mape = args
#         print(f'{dataset_name} - {model_name} - {target}:')
#         print('Mean Squared Error:', mse)
#         print('R-squared:', r2)
#         print('Mean Absolute Percentage Error (MAPE):', mape)
#         print()
#         print('---------------------------------------------------------------')
#         print()

# Run models and view results

In [ ]:
datasets_names = ['Part_desc', 'Power', 'Cadence', 'VO2', 'Part_desc & Power', 'Part_desc & Power & Cadence', 'Part_desc & Power & Cadence & VO2', 'Part_desc & Power & Variants & Cadence & VO2', 'Part_desc & Peaks & Derivatives']
models_names = ['Baseline', 'Linear', 'Decision Tree', 'Random Forest']

datasets = [part_desc_ready, power, cadence, vo2, part_desc_power_joined, part_desc_power_cadence_joined, part_desc_power_cadence_vo2_joined, part_desc_power_variants_cadence_vo2_joined, part_desc_peaks_derivatives_joined]
models = [baseline, linear_regression, decision_tree_regression, random_forest_regression]
targets = ['CP', 'w']

# Create an empty list to store the results
results_list = []

def add_result_to_list(cv, dataset_name, model_name, target, *args):
    if cv == 'Y':
        mse_test, r2_test, mape_test, mse_cv, r2_cv, mape_cv = args
        result_row = {'Model': model_name, 'Dataset': dataset_name, 'Target': target, 'MSE_TEST': mse_test, 'R2_TEST': r2_test, 'MAPE_TEST': mape_test, 'MSE_CV': mse_cv, 'R2_CV': r2_cv, 'MAPE_CV': mape_cv}
    else:
        mse, r2, mape = args
        result_row = {'Model': model_name, 'Dataset': dataset_name, 'Target': target, 'MSE_TEST': mse, 'R2_TEST': r2, 'MAPE_TEST': mape, 'MSE_CV': None, 'R2_CV': None, 'MAPE_CV': None}
    results_list.append(result_row)

# Loop through each combination and add results to the list
for target in targets:
    if target == 'CP':
        target_name = 'CP'
    elif target == 'w':
        target_name = 'W'
    # Baseline model
    for dataset, dataset_name in zip(datasets, datasets_names):
#         print_results('N', dataset_name, 'Baseline', target_name, *baseline(*train_test_split(dataset, target)))
        add_result_to_list('N', dataset_name, 'Baseline', target_name, *baseline(*train_test_split(dataset, target)))

    # All other models
    for model, model_name in zip(models[1:], models_names[1:]):
        for dataset, dataset_name in zip(datasets, datasets_names):
#             print_results('Y', dataset_name, model_name, target_name, *model(*train_test_split(dataset, target)))
            add_result_to_list('Y', dataset_name, model_name, target_name, *model(*train_test_split(dataset, target)))

# Create a DataFrame from the results list & export to excel
results_df = pd.DataFrame(results_list)
results_df.to_excel('results.xlsx', index=False)
print('Results exported to "results.xlsx"')